<a href="https://colab.research.google.com/github/shuklatushar/shuklatushar/blob/main/bondora.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing dependencies


In [81]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn import metrics
pd.set_option('display.max_columns', 500)
import warnings
warnings.filterwarnings("ignore")

In [82]:
db = pd.read_csv('/content/Bondora_raw.csv')

In [83]:
db.head()

,ReportAsOfEOD,LoanId,LoanNumber,ListedOnUTC,BiddingStartedOn,BidsPortfolioManager,BidsApi,BidsManual,UserName,NewCreditCustomer,LoanApplicationStartedDate,LoanDate,ContractEndDate,FirstPaymentDate,MaturityDate_Original,MaturityDate_Last,ApplicationSignedHour,ApplicationSignedWeekday,VerificationType,LanguageCode,Age,DateOfBirth,Gender,Country,AppliedAmount,Amount,Interest,LoanDuration,MonthlyPayment,County,City,UseOfLoan,Education,MaritalStatus,NrOfDependants,EmploymentStatus,EmploymentDurationCurrentEmployer,EmploymentPosition,WorkExperience,OccupationArea,HomeOwnershipType,IncomeFromPrincipalEmployer,IncomeFromPension,IncomeFromFamilyAllowance,IncomeFromSocialWelfare,IncomeFromLeavePay,IncomeFromChildSupport,IncomeOther,IncomeTotal,ExistingLiabilities,LiabilitiesTotal,RefinanceLiabilities,DebtToIncome,FreeCash,MonthlyPaymentDay,ActiveScheduleFirstPaymentReached,PlannedPrincipalTillDate,PlannedInterestTillDate,LastPaymentOn,CurrentDebtDaysPrimary,DebtOccuredOn,CurrentDebtDaysSecondary,DebtOccuredOnForSecondary,ExpectedLoss,LossGivenDefault,ExpectedReturn,ProbabilityOfDefault,DefaultDate,PrincipalOverdueBySchedule,PlannedPrincipalPostDefault,PlannedInterestPostDefault,EAD1,EAD2,PrincipalRecovery,InterestRecovery,RecoveryStage,StageActiveSince,ModelVersion,Rating,EL_V0,Rating_V0,EL_V1,Rating_V1,Rating_V2,Status,Restructured,ActiveLateCategory,WorseLateCategory,CreditScoreEsMicroL,CreditScoreEsEquifaxRisk,CreditScoreFiAsiakasTietoRiskGrade,CreditScoreEeMini,PrincipalPaymentsMade,InterestAndPenaltyPaymentsMade,PrincipalWriteOffs,InterestAndPenaltyWriteOffs,PrincipalBalance,InterestAndPenaltyBalance,NoOfPreviousLoansBeforeLoan,AmountOfPreviousLoansBeforeLoan,PreviousRepaymentsBeforeLoan,PreviousEarlyRepaymentsBefoleLoan,PreviousEarlyRepaymentsCountBeforeLoan,GracePeriodStart,GracePeriodEnd,NextPaymentDate,NextPaymentNr,NrOfScheduledPayments,ReScheduledOn,PrincipalDebtServicingCost,InterestAndPenaltyDebtServicingCost,ActiveLateLastPaymentCategory
0,2020-01-27,F0660C80-83F3-4A97-8DA0-9C250112D6EC,659,2009-06-11 16:40:39,2009-06-11 16:40:39,0,0,115.0410,KARU,True,2009-06-11 16:40:39,2009-06-16,2010-07-06,2009-07-27,2010-06-25,2010-06-25,17,5,2.0,1,61,1947-11-26,1.0,EE,319.5582,115.0408,30.0,12,NaN,HARJU,TALLINN,7,3.0,1.0,0,3.0,UpTo3Years,klienditeenindaja,MoreThan25Years,7.0,NaN,6000.0,0.0,0.0,0.0,0.0,0.0,4500.0,10500.0,0,0.0,0,0.0,0.0,25,True,1800.0000,319.0800,2010-07-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Repaid,False,NaN,91-120,NaN,NaN,NaN,NaN,115.0408,20.4222,0.0,0.0,0.00,0.00,1,83.0852,0.0000,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN
1,2020-01-27,978BB85B-1C69-4D51-8447-9C240104A3A2,654,2009-06-10 15:48:57,2009-06-10 15:48:57,0,0,140.6057,koort681,False,2009-06-10 15:48:57,2009-06-15,2009-07-07,2009-07-15,2009-07-15,2009-07-15,20,4,2.0,1,48,1960-11-05,1.0,EE,191.7349,140.6057,25.0,1,NaN,IDA-VIRU,KOHTLA-JARVE,7,5.0,4.0,0,3.0,MoreThan5Years,Õppealajuhataja lasteaias,MoreThan25Years,16.0,NaN,8300.0,0.0,0.0,0.0,0.0,0.0,2500.0,10800.0,0,0.0,0,0.0,0.0,15,True,2200.0000,45.8300,2009-07-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Repaid,False,NaN,NaN,NaN,NaN,NaN,NaN,140.6057,2.0227,0.0,0.0,0.00,0.00,2,255.6467,258.6256,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN
2,2020-01-27,EA44027E-7FA7-4BB2-846D-9C1F013C8A22,641,2009-06-05 19:12:29,2009-06-05 19:12:29,0,0,319.5580,0ie,True,2009-06-05 19:12:29,2009-06-15,NaN,2009-07-27,2011-02-25,2014-05-13,20,6,2.0,1,58,1950-11-13,1.0,EE,319.5582,319.5409,25.0,20,NaN,PÄRNU,PÄRNU,2,4.0,1.0,0,3.0,UpTo4Years,teenindaja,MoreThan25Years,9.0,NaN,5000.0,0.0,0.0,0.0,0.0,0.0,2000.0,7000.0,0,0.0,0,0.0,0.0,25,True,319.5409,197.2926,2012-10-01,2813.0,2012-05-14,2935.0,2012-01-13,NaN,NaN,NaN,NaN,2012-07-16,116.35,236.97,38.24,279.5049,270.7323,163.1549,50.99,NaN,2016-03-03 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Late,True,180+,180+,NaN,NaN,NaN,NaN,203.1909,59.7626,0.0,0.0,116.35,414.07,0,0.0000,

In [84]:
db.shape

(134529, 112)

In [85]:
db.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 134529 entries, 0 to 134528
Columns: 112 entries, ReportAsOfEOD to ActiveLateLastPaymentCategory
dtypes: bool(3), float64(56), int64(14), object(39)
memory usage: 112.3+ MB


In [86]:
db.isnull().sum()

ReportAsOfEOD                              0
LoanId                                     0
LoanNumber                                 0
ListedOnUTC                                0
BiddingStartedOn                           0
                                       ...  
NrOfScheduledPayments                  36741
ReScheduledOn                          94207
PrincipalDebtServicingCost             75400
InterestAndPenaltyDebtServicingCost    75400
ActiveLateLastPaymentCategory          82279
Length: 112, dtype: int64

Data cleaning

checking percentage pf missing value

In [87]:
drop_list=db.columns[db.isnull().mean()*100>=50].tolist()

In [88]:
db[drop_list].isnull().mean()*100

ContractEndDate                        56.155922
NrOfDependants                         73.537304
EmploymentPosition                     73.173070
WorkExperience                         72.851950
PlannedPrincipalTillDate               59.518765
CurrentDebtDaysPrimary                 63.934914
DebtOccuredOn                          63.934914
CurrentDebtDaysSecondary               61.160791
DebtOccuredOnForSecondary              61.160791
DefaultDate                            68.099815
PlannedPrincipalPostDefault            68.099815
PlannedInterestPostDefault             68.099815
EAD1                                   68.099815
EAD2                                   68.099815
PrincipalRecovery                      68.099815
InterestRecovery                       68.099815
EL_V0                                  96.603706
Rating_V0                              96.603706
EL_V1                                  90.394636
Rating_V1                              90.394636
Rating_V2           

In [89]:
#now we will drop the columns which contain more than 50% null value
db.drop(columns=drop_list,inplace=True)

In [90]:
db.shape

(134529, 78)

In [91]:
db.columns

Index(['ReportAsOfEOD', 'LoanId', 'LoanNumber', 'ListedOnUTC',
       'BiddingStartedOn', 'BidsPortfolioManager', 'BidsApi', 'BidsManual',
       'UserName', 'NewCreditCustomer', 'LoanApplicationStartedDate',
       'LoanDate', 'FirstPaymentDate', 'MaturityDate_Original',
       'MaturityDate_Last', 'ApplicationSignedHour',
       'ApplicationSignedWeekday', 'VerificationType', 'LanguageCode', 'Age',
       'DateOfBirth', 'Gender', 'Country', 'AppliedAmount', 'Amount',
       'Interest', 'LoanDuration', 'MonthlyPayment', 'County', 'City',
       'UseOfLoan', 'Education', 'MaritalStatus', 'EmploymentStatus',
       'EmploymentDurationCurrentEmployer', 'OccupationArea',
       'HomeOwnershipType', 'IncomeFromPrincipalEmployer', 'IncomeFromPension',
       'IncomeFromFamilyAllowance', 'IncomeFromSocialWelfare',
       'IncomeFromLeavePay', 'IncomeFromChildSupport', 'IncomeOther',
       'IncomeTotal', 'ExistingLiabilities', 'LiabilitiesTotal',
       'RefinanceLiabilities', 'DebtToIncome'

In [92]:
db.drop(columns=['DateOfBirth','UserName','LoanId', 'LoanNumber'],inplace=True)

In [93]:
db.drop(columns=['IncomeFromPrincipalEmployer', 'IncomeFromPension', 'IncomeFromFamilyAllowance', 'IncomeFromSocialWelfare',
                 'IncomeFromLeavePay', 'IncomeFromChildSupport', 'IncomeOther'], inplace = True)

In [94]:
db.columns

Index(['ReportAsOfEOD', 'ListedOnUTC', 'BiddingStartedOn',
       'BidsPortfolioManager', 'BidsApi', 'BidsManual', 'NewCreditCustomer',
       'LoanApplicationStartedDate', 'LoanDate', 'FirstPaymentDate',
       'MaturityDate_Original', 'MaturityDate_Last', 'ApplicationSignedHour',
       'ApplicationSignedWeekday', 'VerificationType', 'LanguageCode', 'Age',
       'Gender', 'Country', 'AppliedAmount', 'Amount', 'Interest',
       'LoanDuration', 'MonthlyPayment', 'County', 'City', 'UseOfLoan',
       'Education', 'MaritalStatus', 'EmploymentStatus',
       'EmploymentDurationCurrentEmployer', 'OccupationArea',
       'HomeOwnershipType', 'IncomeTotal', 'ExistingLiabilities',
       'LiabilitiesTotal', 'RefinanceLiabilities', 'DebtToIncome', 'FreeCash',
       'MonthlyPaymentDay', 'ActiveScheduleFirstPaymentReached',
       'PlannedInterestTillDate', 'LastPaymentOn', 'ExpectedLoss',
       'LossGivenDefault', 'ExpectedReturn', 'ProbabilityOfDefault',
       'PrincipalOverdueBySchedule'

In [95]:
db.head()

,ReportAsOfEOD,ListedOnUTC,BiddingStartedOn,BidsPortfolioManager,BidsApi,BidsManual,NewCreditCustomer,LoanApplicationStartedDate,LoanDate,FirstPaymentDate,MaturityDate_Original,MaturityDate_Last,ApplicationSignedHour,ApplicationSignedWeekday,VerificationType,LanguageCode,Age,Gender,Country,AppliedAmount,Amount,Interest,LoanDuration,MonthlyPayment,County,City,UseOfLoan,Education,MaritalStatus,EmploymentStatus,EmploymentDurationCurrentEmployer,OccupationArea,HomeOwnershipType,IncomeTotal,ExistingLiabilities,LiabilitiesTotal,RefinanceLiabilities,DebtToIncome,FreeCash,MonthlyPaymentDay,ActiveScheduleFirstPaymentReached,PlannedInterestTillDate,LastPaymentOn,ExpectedLoss,LossGivenDefault,ExpectedReturn,ProbabilityOfDefault,PrincipalOverdueBySchedule,RecoveryStage,StageActiveSince,ModelVersion,Rating,Status,Restructured,WorseLateCategory,CreditScoreEsMicroL,CreditScoreEeMini,PrincipalPaymentsMade,InterestAndPenaltyPaymentsMade,PrincipalBalance,InterestAndPenaltyBalance,NoOfPreviousLoansBeforeLoan,AmountOfPreviousLoansBeforeLoan,PreviousRepaymentsBeforeLoan,PreviousEarlyRepaymentsCountBeforeLoan,NextPaymentNr,NrOfScheduledPayments
0,2020-01-27,2009-06-11 16:40:39,2009-06-11 16:40:39,0,0,115.0410,True,2009-06-11 16:40:39,2009-06-16,2009-07-27,2010-06-25,2010-06-25,17,5,2.0,1,61,1.0,EE,319.5582,115.0408,30.0,12,NaN,HARJU,TALLINN,7,3.0,1.0,3.0,UpTo3Years,7.0,NaN,10500.0,0,0.0,0,0.0,0.0,25,True,319.0800,2010-07-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Repaid,False,91-120,NaN,NaN,115.0408,20.4222,0.00,0.00,1,83.0852,0.0000,0,NaN,NaN
1,2020-01-27,2009-06-10 15:48:57,2009-06-10 15:48:57,0,0,140.6057,False,2009-06-10 15:48:57,2009-06-15,2009-07-15,2009-07-15,2009-07-15,20,4,2.0,1,48,1.0,EE,191.7349,140.6057,25.0,1,NaN,IDA-VIRU,KOHTLA-JARVE,7,5.0,4.0,3.0,MoreThan5Years,16.0,NaN,10800.0,0,0.0,0,0.0,0.0,15,True,45.8300,2009-07-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Repaid,False,NaN,NaN,NaN,140.6057,2.0227,0.00,0.00,2,255.6467,258.6256,0,NaN,NaN
2,2020-01-27,2009-06-05 19:12:29,2009-06-05 19:12:29,0,0,319.5580,True,2009-06-05 19:12:29,2009-06-15,2009-07-27,2011-02-25,2014-05-13,20,6,2.0,1,58,1.0,EE,319.5582,319.5409,25.0,20,NaN,PÄRNU,PÄRNU,2,4.0,1.0,3.0,UpTo4Years,9.0,NaN,7000.0,0,0.0,0,0.0,0.0,25,True,197.2926,2012-10-01,NaN,NaN,NaN,NaN,116.35,NaN,2016-03-03 00:00:00,NaN,NaN,Late,True,180+,NaN,NaN,203.1909,59.7626,116.35,414.07,0,0.0000,0.0000,0,NaN,NaN
3,2020-01-27,2009-06-13 12:01:20,2009-06-13 12:01:20,0,0,57.5205,True,2009-06-13 12:01:20,2009-06-15,2009-07-15,2010-09-15,2010-09-15,12,7,2.0,1,23,1.0,EE,127.8233,57.5205,45.0,15,NaN,HARJU,PALDISKI,0,2.0,3.0,NaN,UpTo2Years,1.0,NaN,11600.0,0,0.0,0,0.0,0.0,15,True,293.1000,2010-09-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Repaid,False,31-60,NaN,NaN,57.5205,18.7323,0.00,0.00,1,134.2144,0.0000,0,NaN,NaN
4,2020-01-27,2009-06-10 10:17:13,2009-06-10 10:17:13,0,0,319.5582,True,2009-06-10 10:17:13,2009-06-14,2009-07-27,2010-06-25,2010-06-25,10,4,2.0,1,25,1.0,EE,319.5582,319.5436,30.0,12,NaN,TARTU,TARTU,6,4.0,2.0,3.0,UpTo2Years,7.0,NaN,6800.0,0,0.0,0,0.0,0.0,25,True,833.8100,2015-07-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Repaid,False,180+,NaN,NaN,319.5436,220.4200,0.00,0.00,1,146.9966,0.0000,0,NaN,NaN


In [96]:
#deleting date features
db.drop(columns=['ReportAsOfEOD', 'ListedOnUTC', 'BiddingStartedOn', 'LoanApplicationStartedDate', 'LastPaymentOn',
                 'MonthlyPaymentDay','LoanDate', 'FirstPaymentDate', 'MaturityDate_Original','MaturityDate_Last'], 
        inplace = True)

In [97]:
db.drop(columns=['NextPaymentNr', 'NrOfScheduledPayments', 'ApplicationSignedHour', 'ApplicationSignedWeekday', 
                 'ActiveScheduleFirstPaymentReached', 'PlannedInterestTillDate', 'ExpectedLoss','LossGivenDefault',
                'ExpectedReturn', 'ProbabilityOfDefault', 'PrincipalOverdueBySchedule', 'StageActiveSince', 
                'ModelVersion', 'WorseLateCategory'], inplace = True)

In [98]:
db.head()

,BidsPortfolioManager,BidsApi,BidsManual,NewCreditCustomer,VerificationType,LanguageCode,Age,Gender,Country,AppliedAmount,Amount,Interest,LoanDuration,MonthlyPayment,County,City,UseOfLoan,Education,MaritalStatus,EmploymentStatus,EmploymentDurationCurrentEmployer,OccupationArea,HomeOwnershipType,IncomeTotal,ExistingLiabilities,LiabilitiesTotal,RefinanceLiabilities,DebtToIncome,FreeCash,RecoveryStage,Rating,Status,Restructured,CreditScoreEsMicroL,CreditScoreEeMini,PrincipalPaymentsMade,InterestAndPenaltyPaymentsMade,PrincipalBalance,InterestAndPenaltyBalance,NoOfPreviousLoansBeforeLoan,AmountOfPreviousLoansBeforeLoan,PreviousRepaymentsBeforeLoan,PreviousEarlyRepaymentsCountBeforeLoan
0,0,0,115.0410,True,2.0,1,61,1.0,EE,319.5582,115.0408,30.0,12,NaN,HARJU,TALLINN,7,3.0,1.0,3.0,UpTo3Years,7.0,NaN,10500.0,0,0.0,0,0.0,0.0,NaN,NaN,Repaid,False,NaN,NaN,115.0408,20.4222,0.00,0.00,1,83.0852,0.0000,0
1,0,0,140.6057,False,2.0,1,48,1.0,EE,191.7349,140.6057,25.0,1,NaN,IDA-VIRU,KOHTLA-JARVE,7,5.0,4.0,3.0,MoreThan5Years,16.0,NaN,10800.0,0,0.0,0,0.0,0.0,NaN,NaN,Repaid,False,NaN,NaN,140.6057,2.0227,0.00,0.00,2,255.6467,258.6256,0
2,0,0,319.5580,True,2.0,1,58,1.0,EE,319.5582,319.5409,25.0,20,NaN,PÄRNU,PÄRNU,2,4.0,1.0,3.0,UpTo4Years,9.0,NaN,7000.0,0,0.0,0,0.0,0.0,NaN,NaN,Late,True,NaN,NaN,203.1909,59.7626,116.35,414.07,0,0.0000,0.0000,0
3,0,0,57.5205,True,2.0,1,23,1.0,EE,127.8233,57.5205,45.0,15,NaN,HARJU,PALDISKI,0,2.0,3.0,NaN,UpTo2Years,1.0,NaN,11600.0,0,0.0,0,0.0,0.0,NaN,NaN,Repaid,False,NaN,NaN,57.5205,18.7323,0.00,0.00,1,134.2144,0.0000,0
4,0,0,319.5582,True,2.0,1,25,1.0,EE,319.5582,319.5436,30.0,12,NaN,TARTU,TARTU,6,4.0,2.0,3.0,UpTo2Years,7.0,NaN,6800.0,0,0.0,0,0.0,0.0,NaN,NaN,Repaid,False,NaN,NaN,319.5436,220.4200,0.00,0.00,1,146.9966,0.0000,0


In [99]:
db.shape

(134529, 43)

**Creating Target Variable**

Here, status is the variable which help us in creating target variable. The reason for not making status as target variable is that it has three unique values current, Late and repaid. There is no default feature but there is a feature default date which tells us when the borrower has defaulted means on which date the borrower defaulted. So, we will be combining Status and Default date features for creating target variable.The reason we cannot simply treat Late as default because it also has some records in which actual status is Late but the user has never defaulted i.e., default date is null. So we will first filter out all the current status records because they are not matured yet they are current loans.

Now, we will create new target variable in which 0 will be assigned when default date is null means borrower has never defaulted while 1 in case default date is present.

In [100]:
# Add DefaultDate Column to our df dataset
db["DefaultDate"] = pd.read_csv('/content/Bondora_raw.csv',low_memory=False).DefaultDate

In [101]:
# let's find the counts of each status categories 
db.Status.value_counts()

Current    57135
Late       45772
Repaid     31622
Name: Status, dtype: int64

In [102]:
# filtering out Current Status records
db = db[db.Status != 'Current']

In [103]:
db.Status.value_counts()

Late      45772
Repaid    31622
Name: Status, dtype: int64

In [104]:
db.shape

(77394, 44)

In [105]:
# Rplace any null value by NoDefault in DefaultDate column
db.DefaultDate.fillna('NoDefault', inplace = True)

In [106]:
# Replace any Date by Defalut
for i in range(len(db.DefaultDate.values)):
    if db.DefaultDate.values[i] != 'NoDefault':
        db.DefaultDate.values[i] = 'Default'

In [107]:
# check the counts of default and nondefault 
db.DefaultDate.value_counts()

Default      42794
NoDefault    34600
Name: DefaultDate, dtype: int64

In [108]:
# let's drop the status columns
db.drop(columns=['Status'], inplace = True)

In [109]:
# Rename DefaultDate column to LoanStatus
db.rename(columns={'DefaultDate':'LoanStatus'}, inplace = True)

Checking datatype of all features
In this step we will see any data type mismatch

In [110]:
# write your code here
db.dtypes

BidsPortfolioManager                        int64
BidsApi                                     int64
BidsManual                                float64
NewCreditCustomer                            bool
VerificationType                          float64
LanguageCode                                int64
Age                                         int64
Gender                                    float64
Country                                    object
AppliedAmount                             float64
Amount                                    float64
Interest                                  float64
LoanDuration                                int64
MonthlyPayment                            float64
County                                     object
City                                       object
UseOfLoan                                   int64
Education                                 float64
MaritalStatus                             float64
EmploymentStatus                          float64


As we can see in numeric column distribution there are many columns which are present as numeric but they are actually categorical as per data description such as Verification Type, Language Code, Gender, Use of Loan, Education, Marital Status,EmployementStatus, OccupationArea etc.
So we will convert these features to categorical features as with numerical value we can assign value to feature
For verification column:

In [111]:
# Replace Null values by 'Not Set'
db.VerificationType.fillna('Not Set', inplace = True)

In [112]:
# convert from float type to string first
db['VerificationType'] = db['VerificationType'].astype(object)

for i in range(len(db.VerificationType.values)):
    db.VerificationType.values[i] = str(db.VerificationType.values[i])

In [113]:
# From Bonora dataser we can find that:
# 0 Not set, 1 Income unverified, 2 Income unverified, cross-referenced by phone, 3 Income verified 
db.VerificationType = db.VerificationType.str.replace('0.0', 'Not Set')
db.VerificationType = db.VerificationType.str.replace('1.0', 'Income unverified')
db.VerificationType = db.VerificationType.str.replace('2.0', 'Income unverified,cross-referenced by phone')
db.VerificationType = db.VerificationType.str.replace('3.0', 'Income verified')
db.VerificationType = db.VerificationType.str.replace('4.0', 'Income and expenses verified')

In [114]:
db.VerificationType.value_counts()

Income and expenses verified                   41428
Income unverified                              25626
Income verified                                 8460
Income unverified,cross-referenced by phone     1827
Not Set                                           53
Name: VerificationType, dtype: int64

In [115]:
db.Gender.value_counts()

0.0    50697
1.0    20927
2.0     5725
Name: Gender, dtype: int64

In [116]:
# Replace Null values by Undefined
db.Gender.fillna('Undefined', inplace = True)

In [117]:
# Convert from int types to string first
db['Gender'] = db['Gender'].astype(object)

for i in range(len(db.Gender.values)):
    db.Gender.values[i] = str(db.Gender.values[i])

In [118]:
# From Bonora dataser we can find that:
## 0.0 Male 1.0 Female 2.0 Undefined
db.Gender = db.Gender.str.replace('0.0', 'Male')
db.Gender = db.Gender.str.replace('1.0', 'Female')
db.Gender = db.Gender.str.replace('2.0', 'Undefined')

In [119]:
# Check
db.Gender.value_counts()


Male         50697
Female       20927
Undefined     5770
Name: Gender, dtype: int64

lets handle language column


In [124]:
db.LanguageCode=db.LanguageCode.astype(object)
for i in range(len(db.LanguageCode.values)):
    db.LanguageCode.values[i] = str(db.LanguageCode.values[i])


In [126]:
# From Bonora dataser we can find that:
## 1 Estonian 2 English 3 Russian 4 Finnish 5 German 6 Spanish 9 Slovakian
### We will give 'Other' to other Language code
#### You should handle 'Other' first
db.LanguageCode = db.LanguageCode.str.replace('7',  'Other')
db.LanguageCode = db.LanguageCode.str.replace('10', 'Other')
db.LanguageCode = db.LanguageCode.str.replace('13', 'Other')
db.LanguageCode = db.LanguageCode.str.replace('15', 'Other')
db.LanguageCode = db.LanguageCode.str.replace('21', 'Other')
db.LanguageCode = db.LanguageCode.str.replace('22', 'Other')
db.LanguageCode = db.LanguageCode.str.replace('1', 'Estonian')
db.LanguageCode = db.LanguageCode.str.replace('2', 'English')
db.LanguageCode = db.LanguageCode.str.replace('3', 'Russian')
db.LanguageCode = db.LanguageCode.str.replace('4', 'Finnish')
db.LanguageCode = db.LanguageCode.str.replace('5', 'German')
db.LanguageCode = db.LanguageCode.str.replace('6', 'Spanish')
db.LanguageCode = db.LanguageCode.str.replace('9', 'Slovakian')

In [128]:
# Convert from int types to string first
db.UseOfLoan = db.UseOfLoan.astype(object)

for i in range(len(db.UseOfLoan.values)):
    db.UseOfLoan.values[i] = str(db.UseOfLoan.values[i])

In [129]:
# From Bondora dataset and Bondora Statistics Page we can find that:
## -1 Not Set, 0 Loan consolidation , 1 Real estate, 2 Home improvement, 3 Business, 4 Education, 5 Travel, 6 Vehicle, 7 Other,
## 8 Health, 102  Purchase of machinery equipment, 110 Other business, 104 Accounts receivable financing, 
## 101 Working capital financing, 107 Acquisition of stocks, 108 Acquisition of real estate, 106 Acquisition of means of transport

db.UseOfLoan = db.UseOfLoan.str.replace('102', 'Purchase of machinery equipment')
db.UseOfLoan = db.UseOfLoan.str.replace('110', 'Other business')
db.UseOfLoan = db.UseOfLoan.str.replace('104', 'Accounts receivable financing')
db.UseOfLoan = db.UseOfLoan.str.replace('101', 'Working capital financing')
db.UseOfLoan = db.UseOfLoan.str.replace('107', 'Acquisition of stocks')
db.UseOfLoan = db.UseOfLoan.str.replace('106', 'Acquisition of means of transport')
db.UseOfLoan = db.UseOfLoan.str.replace('108', 'Acquisition of real estate')

db.UseOfLoan = db.UseOfLoan.str.replace('-1',  'Not Set')
db.UseOfLoan = db.UseOfLoan.str.replace('0',   'Loan consolidation')
db.UseOfLoan = db.UseOfLoan.str.replace('1',   'Real estate')
db.UseOfLoan = db.UseOfLoan.str.replace('2',   'Home improvement')
db.UseOfLoan = db.UseOfLoan.str.replace('3',   'Business')
db.UseOfLoan = db.UseOfLoan.str.replace('4',   'Education')
db.UseOfLoan = db.UseOfLoan.str.replace('5',   'Travel')
db.UseOfLoan = db.UseOfLoan.str.replace('6',   'Vehicle')
db.UseOfLoan = db.UseOfLoan.str.replace('7',   'Other')
db.UseOfLoan = db.UseOfLoan.str.replace('8',   'Health')

In [130]:
# We will replace Null values
db.Education.fillna('Not Present', inplace = True)

In [131]:
db.Education.value_counts()

4.0            28557
5.0            20919
3.0            17257
2.0             5379
1.0             5226
Not Present       45
0.0                8
-1.0               3
Name: Education, dtype: int64

In [132]:
# Convert from int types to string first
db.Education = db.Education.astype(object)

for i in range(len(db.Education.values)):
    db.Education.values[i] = str(db.Education.values[i])

In [133]:
# We will replace 0 and -1 by "Not Present".
## And from Bondora dataset and Bondora Statistics Page we can find that:
## 1 Primary education, 2 Basic education, 3 Vocational education, 4 Secondary education, 5 Higher education.
db.Education = db.Education.str.replace('-1.0',  'Not Present')
db.Education = db.Education.str.replace('0.0',   'Not Present')
db.Education = db.Education.str.replace('1.0',   'Primary education')
db.Education = db.Education.str.replace('2.0',   'Basic education')
db.Education = db.Education.str.replace('3.0',   'Vocational education')
db.Education = db.Education.str.replace('4.0',   'Secondary education')
db.Education = db.Education.str.replace('5.0',   'Higher education')

Let's handle MaritalStatus column :

In [135]:
db.MaritalStatus.value_counts()


-1.0    44177
 3.0    11504
 1.0     9585
 2.0     8479
 4.0     3077
 5.0      519
 0.0        8
Name: MaritalStatus, dtype: int64

In [136]:
# Replace Null values by "Not Specified"
db.MaritalStatus.fillna('Not Specified', inplace = True)


In [137]:
# Convert from int types to string first
db.MaritalStatus = db.MaritalStatus.astype(object)

for i in range(len(db.MaritalStatus.values)):
    db.MaritalStatus.values[i] = str(db.MaritalStatus.values[i])

In [139]:
# Again Marital status of value 0 and -1 has no description so we will encode them as Not_specified
## And from Bondora dataser:
## 1 Married 2 Cohabitant 3 Single 4 Divorced 5 Widow
db.MaritalStatus = db.MaritalStatus.str.replace('-1.0',  'Not Specified')
db.MaritalStatus = db.MaritalStatus.str.replace('0.0',   'Not Specified')
db.MaritalStatus = db.MaritalStatus.str.replace('1.0',   'Married')
db.MaritalStatus = db.MaritalStatus.str.replace('2.0',   'Cohabitant')
db.MaritalStatus = db.MaritalStatus.str.replace('3.0',   'Single')
db.MaritalStatus = db.MaritalStatus.str.replace('4.0',   'Divorced')
db.MaritalStatus = db.MaritalStatus.str.replace('5.0',   'Widow')

In [141]:
db.EmploymentStatus.fillna('Not present', inplace = True)

In [142]:
# Convert from int types to string first
db.EmploymentStatus = db.EmploymentStatus.astype(object)

for i in range(len(db.EmploymentStatus.values)):
    db.EmploymentStatus.values[i] = str(db.EmploymentStatus.values[i])

In [143]:
# Again EmploymentStatus of value 0 and -1 has no description so we will encode them as Not present
## And from Bondora dataser:
## 1 Unemployed 2 Partially employed 3 Fully employed 4 Self-employed 5 Entrepreneur 6 Retiree
db.EmploymentStatus = db.EmploymentStatus.str.replace('-1.0',  'Not present')
db.EmploymentStatus = db.EmploymentStatus.str.replace('0.0',   'Not present')
db.EmploymentStatus = db.EmploymentStatus.str.replace('2.0',   'Partially employed')
db.EmploymentStatus = db.EmploymentStatus.str.replace('3.0',   'Fully employed')
db.EmploymentStatus = db.EmploymentStatus.str.replace('4.0',   'Self-employed')
db.EmploymentStatus = db.EmploymentStatus.str.replace('5.0',   'Entrepreneur')
db.EmploymentStatus = db.EmploymentStatus.str.replace('6.0',   'Retiree')

In [144]:

db.EmploymentStatus.value_counts()

Not present           44406
Fully employed        27309
Entrepreneur           1695
Retiree                1663
Self-employed          1211
Partially employed     1110
Name: EmploymentStatus, dtype: int64

In [146]:
db.NewCreditCustomer.value_counts()

True     48867
False    28527
Name: NewCreditCustomer, dtype: int64

In [147]:
# Convert from int types to string first
db.NewCreditCustomer = db.NewCreditCustomer.astype(object)

for i in range(len(db.NewCreditCustomer.values)):
    db.NewCreditCustomer.values[i] = str(db.NewCreditCustomer.values[i])

In [148]:
db.Restructured.value_counts()

False    61885
True     15509
Name: Restructured, dtype: int64

In [149]:
# Convert from int types to string first
db.Restructured = db.Restructured.astype(object)

for i in range(len(db.Restructured.values)):
    db.Restructured.values[i] = str(db.Restructured.values[i])

In [150]:
# Replace Null values by "Not present"
db.OccupationArea.fillna('Not present', inplace = True)

In [152]:
db.OccupationArea.value_counts()

-1.0           44228
1.0             7640
7.0             3207
6.0             3007
3.0             2876
8.0             2276
17.0            2186
9.0             2106
10.0            1782
15.0            1555
16.0            1251
11.0            1054
19.0             893
14.0             798
4.0              530
18.0             524
13.0             493
12.0             433
5.0              342
2.0              116
Not present       86
0.0               11
Name: OccupationArea, dtype: int64

In [154]:
# Convert from int types to string first
db.OccupationArea = db.OccupationArea.astype(object)

for i in range(len(db.OccupationArea.values)):
    db.OccupationArea.values[i] = str(db.OccupationArea.values[i])

In [155]:
## We will replace -1 and 0 by "Not present"
## And from the Bonora dataset we find : 1 Other 2 Mining 3 Processing 4 Energy 5 Utilities 6 Construction 
## 7 Retail and wholesale 8 Transport and warehousing 9 Hospitality and catering 10 Info and telecom 
## 11 Finance and insurance 12 Real-estate 13 Research 14 Administrative 15 Civil service & military 16 Education 17 
## Healthcare and social help 18 Art and entertainment 19 Agriculture, forestry and fishing
db.OccupationArea = db.OccupationArea.str.replace('10.0', 'Info and telecom')
db.OccupationArea = db.OccupationArea.str.replace('11.0', 'Finance and insurance')
db.OccupationArea = db.OccupationArea.str.replace('12.0', 'Real-estate')
db.OccupationArea = db.OccupationArea.str.replace('13.0', 'Research')
db.OccupationArea = db.OccupationArea.str.replace('14.0', 'Administrative')
db.OccupationArea = db.OccupationArea.str.replace('15.0', 'Civil service & military')
db.OccupationArea = db.OccupationArea.str.replace('16.0', 'Education')
db.OccupationArea = db.OccupationArea.str.replace('17.0', 'Healthcare and social help')
db.OccupationArea = db.OccupationArea.str.replace('18.0', 'Art and entertainment')
db.OccupationArea = db.OccupationArea.str.replace('19.0', 'Agriculture, forestry and fishing')
db.OccupationArea = db.OccupationArea.str.replace('-1.0', 'Not present')
db.OccupationArea = db.OccupationArea.str.replace('0.0',  'Not present')
db.OccupationArea = db.OccupationArea.str.replace('1.0', 'Other')
db.OccupationArea = db.OccupationArea.str.replace('2.0', 'Mining')
db.OccupationArea = db.OccupationArea.str.replace('3.0', 'Processing')
db.OccupationArea = db.OccupationArea.str.replace('4.0', 'Energy')
db.OccupationArea = db.OccupationArea.str.replace('5.0', 'Utilities')
db.OccupationArea = db.OccupationArea.str.replace('6.0', 'Construction')
db.OccupationArea = db.OccupationArea.str.replace('7.0', 'Retail and wholesale')
db.OccupationArea = db.OccupationArea.str.replace('8.0', 'Transport and warehousing')
db.OccupationArea = db.OccupationArea.str.replace('9.0',  'Hospitality and catering')

In [156]:
# Check
db.OccupationArea.value_counts()

Not present                          44325
Other                                 7640
Retail and wholesale                  3207
Construction                          3007
Processing                            2876
Transport and warehousing             2276
Healthcare and social help            2186
Hospitality and catering              2106
Info and telecom                      1782
Civil service & military              1555
Education                             1251
Finance and insurance                 1054
Agriculture, forestry and fishing      893
Administrative                         798
Energy                                 530
Art and entertainment                  524
Research                               493
Real-estate                            433
Utilities                              342
Mining                                 116
Name: OccupationArea, dtype: int64

In [158]:
db.HomeOwnershipType.value_counts()

 1.0     24594
 3.0     16842
 2.0     12776
 8.0      8762
 4.0      4031
 10.0     3179
 7.0      2701
 6.0      1362
 5.0       882
 9.0       565
 0.0        46
-1.0         2
Name: HomeOwnershipType, dtype: int64

In [159]:
# Replcae Null values by "Not specified"
db.HomeOwnershipType.fillna("Not specified", inplace = True)

In [160]:
# Convert from int types to string first
db.HomeOwnershipType = db.HomeOwnershipType.astype(object)

for i in range(len(db.HomeOwnershipType.values)):
    db.HomeOwnershipType.values[i] = str(db.HomeOwnershipType.values[i])

In [161]:
## We will replace -1 and 0 by "Not specified"
## And from the Bonora dataset we find : 0 Homeless 1 Owner 2 Living with parents 3 Tenant,pre-furnished property 
##4 Tenant,unfurnished property 5 Council house 6 Joint tenant 7 Joint ownership 8 Mortgage 9 Owner with encumbrance 10 Other
db.HomeOwnershipType = db.HomeOwnershipType.str.replace('10.0', 'Other')
db.HomeOwnershipType = db.HomeOwnershipType.str.replace('-1.0', 'Not specified')
db.HomeOwnershipType = db.HomeOwnershipType.str.replace('0.0',  'Homeless')
db.HomeOwnershipType = db.HomeOwnershipType.str.replace('1.0', 'Owner')
db.HomeOwnershipType = db.HomeOwnershipType.str.replace('2.0', 'Living with parents')
db.HomeOwnershipType = db.HomeOwnershipType.str.replace('3.0', 'Tenant,pre-furnished property')
db.HomeOwnershipType = db.HomeOwnershipType.str.replace('4.0', 'Tenant,pre-furnished property')
db.HomeOwnershipType = db.HomeOwnershipType.str.replace('5.0', 'Council house')
db.HomeOwnershipType = db.HomeOwnershipType.str.replace('6.0', 'Joint tenant')
db.HomeOwnershipType = db.HomeOwnershipType.str.replace('7.0', 'Joint ownership')
db.HomeOwnershipType = db.HomeOwnershipType.str.replace('8.0', 'Mortgage')
db.HomeOwnershipType = db.HomeOwnershipType.str.replace('9.0',  'Owner with encumbrance')

In [162]:
# Check 
db.HomeOwnershipType.value_counts()

Owner                            24594
Tenant,pre-furnished property    20873
Living with parents              12776
Mortgage                          8762
Other                             3179
Joint ownership                   2701
Not specified                     1654
Joint tenant                      1362
Council house                      882
Owner with encumbrance             565
Homeless                            46
Name: HomeOwnershipType, dtype: int64

In [164]:
# replace Null values by most frequent value
db.Rating.fillna('F', inplace = True)

In [166]:
db.CreditScoreEsMicroL.fillna('M', inplace = True)

In [167]:
db.CreditScoreEsMicroL.fillna('M', inplace = True)

In [168]:
db.EmploymentDurationCurrentEmployer.value_counts()

MoreThan5Years    29328
UpTo1Year         14288
UpTo5Years        13777
UpTo2Years         5541
UpTo3Years         4539
Retiree            3344
UpTo4Years         3057
Other              2027
TrialPeriod         619
Name: EmploymentDurationCurrentEmployer, dtype: int64

In [169]:
db.MonthlyPayment.fillna(db.MonthlyPayment.mean(), inplace = True)
db.DebtToIncome.fillna(db.DebtToIncome.mean(), inplace = True)
db.FreeCash.fillna(db.FreeCash.mean(), inplace = True) 
db.PreviousRepaymentsBeforeLoan.fillna(db.PreviousRepaymentsBeforeLoan.mean(), inplace = True)